In [1]:
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urlparse
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm
import threading

headers = {
    'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'
}

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-software-rasterizer')

session = requests.Session()

start_page = 1
total_pages = 2089
# total_pages = 417
product_pages = []
lock = threading.Lock()

def fetch_product_urls(page):
    try:
        url = f'https://search.shopping.naver.com/search/all?adQuery=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%98%81%EC%96%91%EC%A0%9C&frm=NVSHATC&npayType=2&origQuery=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%98%81%EC%96%91%EC%A0%9C&pagingIndex={page}&pagingSize=40&productSet=checkout&query=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%98%81%EC%96%91%EC%A0%9C&sort=review&timestamp=&viewType=image'
        
        # url = f'https://search.shopping.naver.com/search/all?adQuery=%EA%B3%A0%EC%96%91%EC%9D%B4%20%EC%98%81%EC%96%91%EC%A0%9C&frm=NVSHATC&npayType=2&origQuery=%EA%B3%A0%EC%96%91%EC%9D%B4%20%EC%98%81%EC%96%91%EC%A0%9C&pagingIndex={page}&pagingSize=40&productSet=checkout&query=%EA%B3%A0%EC%96%91%EC%9D%B4%20%EC%98%81%EC%96%91%EC%A0%9C&sort=review&timestamp=&viewType=image'

        driver = webdriver.Chrome(options=chrome_options)

        driver.get(url)
        driver.implicitly_wait(5)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.implicitly_wait(5)

        full_html = driver.page_source

        driver.quit()

        soup = BeautifulSoup(full_html, 'html.parser')

        links = soup.select('div.thumbnail_thumb_wrap__RbcYO._wrapper > a')

        with lock:
            for link in links:
                product_pages.append(link['href'])

    except Exception as e:
        print(f"Error in thread {threading.current_thread().name} on page {page}: {e}")
        pass

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(fetch_product_urls, range(start_page, total_pages+1)), total=total_pages, desc='Fetching product URLs'))

print(len(product_pages))

In [3]:
def download_images(product_page):
    try:
        driver = webdriver.Chrome(options=chrome_options)

        driver.get(product_page)
        driver.implicitly_wait(5)

        driver.find_element(By.CSS_SELECTOR, 'div._3osy73V_eD._1Hc_ju_IXp > button').click()
        driver.implicitly_wait(5)

        full_html = driver.page_source

        driver.quit()

        soup = BeautifulSoup(full_html, 'html.parser')

        product_name = soup.select_one('h3._22kNQuEXmb._copyable').text

        valid_product_name = ''.join(c for c in os.path.splitext(product_name)[0] if c.isalnum() or c in (' ', '_')).rstrip()
        valid_product_name = valid_product_name.replace(' ', '_')
        target_directory = os.path.abspath(f'naver/{valid_product_name}')

        os.makedirs(target_directory, exist_ok=True)

        image_tags = soup.select('.se-main-container img')

        for i, img in enumerate(image_tags):
            if 'data-src' in img.attrs and img['data-src'].strip():
                image_url = img['data-src']

                parsed_url = urlparse(image_url)
                filename, extension = os.path.splitext(os.path.basename(parsed_url.path))

                file_path = os.path.join(target_directory, f'image_{i}{extension}')

                response = session.get(image_url)

                with open(file_path, 'wb') as f:
                    f.write(response.content)

    except Exception as e:
        pass

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(download_images, product_pages), total=len(product_pages), desc='Downloading images'))

20
